# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [3]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)


1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [6]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [7]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

In [10]:
import dlt

#1

In [5]:
generator = square_root_generator(limit)

out_sum = 0
for sqrt_value in generator:
    out_sum += sqrt_value

print('Sum of outputs: ', out_sum)

generator13 = square_root_generator(limit=13)

for sqrt_value in generator13:
    pass

print('13th number yielded: ', sqrt_value)

Sum of outputs:  8.382332347441762
13th number yielded:  3.605551275463989


#2

In [12]:
pipeline = dlt.pipeline(destination='duckdb', dataset_name='people')

# run the pipeline with default settings, and capture the outcome
info = pipeline.run(people_1(),
					table_name="census",
					write_disposition="replace")

# show the outcome
print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.26 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1709548166.0809767 is LOADED and contains no failed jobs


In [14]:
# show outcome

import duckdb

conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n census table below:")

persons1 = conn.sql("SELECT * FROM census").df()
display(persons1)

print("\n\n\n sum of ages:")

sum1 = conn.sql("SELECT SUM(age) FROM census").df()
display(sum1)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ census              │
└─────────────────────┘




 census table below:


,id,name,age,city,_dlt_load_id,_dlt_id
0,1,Person_1,26,City_A,1709548166.0809767,JX0xPkvTPnHvLA
1,2,Person_2,27,City_A,1709548166.0809767,DzMw/2Ok4T0y6Q
2,3,Person_3,28,City_A,1709548166.0809767,xMEqsNug7Tiihg
3,4,Person_4,29,City_A,1709548166.0809767,/SahDYCl6tE2CQ
4,5,Person_5,30,City_A,1709548166.0809767,WQQxlcGgy6uDAA





 sum of ages:


,sum(age)
0,140.0


In [15]:
info = pipeline.run(people_2(),
					table_name="census",
					write_disposition="append")

# show the outcome
print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.23 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1709548923.7561707 is LOADED and contains no failed jobs


In [16]:
print("\n\n\n census table below:")

persons_all = conn.sql("SELECT * FROM census").df()
display(persons_all)

print("\n\n\n sum of ages:")

sum_all = conn.sql("SELECT SUM(age) FROM census").df()
display(sum_all)




 census table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1709548166.0809767,JX0xPkvTPnHvLA,None
1,2,Person_2,27,City_A,1709548166.0809767,DzMw/2Ok4T0y6Q,None
2,3,Person_3,28,City_A,1709548166.0809767,xMEqsNug7Tiihg,None
3,4,Person_4,29,City_A,1709548166.0809767,/SahDYCl6tE2CQ,None
4,5,Person_5,30,City_A,1709548166.0809767,WQQxlcGgy6uDAA,None
5,3,Person_3,33,City_B,1709548923.7561707,LSR3KCRamSwDwA,Job_3
6,4,Person_4,34,City_B,1709548923.7561707,Wv8q2jjqE94SdA,Job_4
7,5,Person_5,35,City_B,1709548923.7561707,vOXc6/AXVpDXlQ,Job_5
8,6,Person_6,36,City_B,1709548923.7561707,H4fm1cRKz0Qv8w,Job_6
9,7,Person_7,37,City_B,1709548923.7561707,S1sfa/LabISz6Q,Job_7





 sum of ages:


,sum(age)
0,353.0


#3

In [17]:
info = pipeline.run(people_1(),
					table_name="census_merged",
					write_disposition="replace",
          primary_key='id')

# show the outcome
print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.24 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1709549344.3831427 is LOADED and contains no failed jobs


In [20]:
info = pipeline.run(people_2(),
					table_name="census_merged",
					write_disposition="merge",
          primary_key="id")

# show the outcome
print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.44 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1709549805.8398206 is LOADED and contains no failed jobs


In [21]:
print("\n\n\n census table below:")

persons_all = conn.sql("SELECT * FROM census_merged").df()
display(persons_all)

print("\n\n\n sum of ages:")

sum_all = conn.sql("SELECT SUM(age) FROM census_merged").df()
display(sum_all)




 census table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1709549344.3831427,cQ/MlTdtcHKf5w,None
1,2,Person_2,27,City_A,1709549344.3831427,WsHQpJRCufOeRQ,None
2,5,Person_5,35,City_B,1709549805.8398206,7m6DB9/dRy5Iyw,Job_5
3,8,Person_8,38,City_B,1709549805.8398206,uSmDteQW2b3DhQ,Job_8
4,7,Person_7,37,City_B,1709549805.8398206,H5bw5x3Hiq82AQ,Job_7
5,4,Person_4,34,City_B,1709549805.8398206,wLCDZiS6CyBpSg,Job_4
6,3,Person_3,33,City_B,1709549805.8398206,4L0AmbUGksCqCg,Job_3
7,6,Person_6,36,City_B,1709549805.8398206,+WR8sGvpnlbv6Q,Job_6





 sum of ages:


,sum(age)
0,266.0


Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX